In [1]:
%load_ext rustdef

load rustdef


<IPython.core.display.Javascript object>

     Created library package


    Updating 'https://github.com/rust-lang/crates.io-index' index
      Adding pyo3 v0.15.0 to dependencies with features: ["extension-module"]


In [2]:
%rustdef deps add --help

usage: %rustdef deps add [-h] args [args ...]

Add dependencies

positional arguments:
  args        Dependencies to be added. See cargo-edit

optional arguments:
  -h, --help  show this help message and exit


In [3]:
%rustdef deps add ndarray@0.15.0 numpy@0.15.0

    Updating 'https://github.com/rust-lang/crates.io-index' index
      Adding ndarray v0.15.0 to dependencies
      Adding numpy v0.15.0 to dependencies


In [4]:
%rustdef deps show

ndarray = "0.15.0"
numpy = "0.15.0"

[pyo3]
version = "0.15.0"
features = [ "extension-module",]



In [5]:
%%rustdef
use numpy::{IntoPyArray, PyArrayDyn, PyReadonlyArrayDyn};
use ndarray::{ArrayD, ArrayViewD};

// immutable example
fn axpy_rs(a: f64, x: ArrayViewD<'_, f64>, y: ArrayViewD<'_, f64>) -> ArrayD<f64> {
    a * &x + &y
}

#[pyfn(m, "axpy")]
fn axpy<'py>(
    py: Python<'py>,
    a: f64,
    x: PyReadonlyArrayDyn<'py, f64>,
    y: PyReadonlyArrayDyn<'py, f64>,
) -> &'py PyArrayDyn<f64> {
    let x = x.as_array();
    let y = y.as_array();
    axpy_rs(a, x, y).into_pyarray(py)
}

Building..
🔗 Found pyo3 bindings
🐍 Found CPython 3.7m at /usr/local/anaconda3/envs/rustdef/bin/python
   Compiling proc-macro2 v1.0.32
   Compiling unicode-xid v0.2.2
   Compiling autocfg v1.0.1
   Compiling proc-macro-hack v0.5.19
   Compiling pyo3-build-config v0.15.0
   Compiling syn v1.0.81
   Compiling once_cell v1.8.0
   Compiling libc v0.2.107
   Compiling parking_lot_core v0.8.5
   Compiling cfg-if v1.0.0
   Compiling scopeguard v1.1.0
   Compiling unindent v0.1.7
   Compiling smallvec v1.7.0
   Compiling rawpointer v0.2.1
   Compiling cfg-if v0.1.10
   Compiling instant v0.1.12           ] 0/51: cfg-if, autocfg, cfg-if, ...
   Compiling matrixmultiply v0.3.1     ] 2/51: autocfg, once_cell, scope......
   Compiling lock_api v0.4.5           ] 3/51: autocfg, once_cell, matri......
   Compiling num-traits v0.2.14          ] 10/51: autocfg, matrixmultiply, ...
   Compiling num-integer v0.1.44      ] 11/51: num-traits(build.rs), mat...
   Compiling quote v1.0.10            ] 25/51:

In [6]:
import numpy as np

In [7]:
axpy(2.4, np.array([1.0, 3.2, 2.9]), np.array([2.3, 10.9, -2]))

array([ 4.7 , 18.58,  4.96])

In [8]:
%%rustdef
use numpy::{IntoPyArray, PyArrayDyn, PyReadonlyArrayDyn};
use ndarray::{ArrayD, ArrayViewD};

// mutable example
#[pyfn(m, "mult_inline")]
fn mult_inline<'py>(_py: Python<'py>, a: f64, x: &'py PyArrayDyn<f64>) -> PyResult<()> {
    let mut x = unsafe { x.as_array_mut() };
    x *= a;
    Ok(())
}

Building..
🔗 Found pyo3 bindings
🐍 Found CPython 3.7m at /usr/local/anaconda3/envs/rustdef/bin/python
   Compiling rustdef_cell_eb290401c4e6e3348fa7a25c89d3386163589099 v0.1.0 (/Users/ryosuke.kamesawa.mot/.rustdef/rustdef_cell_eb290401c4e6e3348fa7a25c89d3386163589099)
  --> rustdef_cell_eb290401c4e6e3348fa7a25c89d3386163589099/src/lib.rs:12:11
   |
12 | #[pyfn(m, "mult_inline")]
   |           ^^^^^^^^^^^^^
   |
   = note: `#[warn(deprecated)]` on by default




    Finished dev [unoptimized + debuginfo] target(s) in 1.89sl_eb290401c4e6...
📦 Built wheel for CPython 3.7m to /Users/ryosuke.kamesawa.mot/.rustdef/target/wheels/rustdef_cell_eb290401c4e6e3348fa7a25c89d3386163589099-0.1.0-cp37-cp37m-macosx_10_7_x86_64.whl
Processing /Users/ryosuke.kamesawa.mot/.rustdef/target/wheels/rustdef_cell_eb290401c4e6e3348fa7a25c89d3386163589099-0.1.0-cp37-cp37m-macosx_10_7_x86_64.whl


In [9]:
x = np.random.rand(10)
y = np.random.rand(10)
x, y

(array([0.78554241, 0.86826045, 0.0069701 , 0.51508886, 0.49016607,
        0.85396403, 0.0254975 , 0.12552982, 0.67412181, 0.32502211]),
 array([0.1188691 , 0.40054706, 0.67865726, 0.20426603, 0.17742266,
        0.24854702, 0.09243243, 0.48208843, 0.59401011, 0.26364802]))

In [10]:
axpy(3.0, x, y)

array([2.47549633, 3.00532841, 0.69956756, 1.7495326 , 1.64792087,
       2.81043911, 0.16892493, 0.85867787, 2.61637553, 1.23871436])

In [11]:
mult_inline(12.0, x)

In [12]:
x

array([ 9.42650888, 10.4191254 ,  0.0836412 ,  6.18106631,  5.88199286,
       10.24756837,  0.30596999,  1.50635779,  8.0894617 ,  3.90026537])